# Compare classification methods for identifying org. science perspectives in JSTOR articles
## Using balanced samples from hand-labeled set of articles

@author: Jaren Haber, PhD<br>
@coauthors: Prof. Heather Haveman, UC Berkeley; Yoon Sung Hong, Wayfair<br>
@contact: Jaren.Haber@georgetown.edu<br>
@project: Computational Literature Review of Organizational Scholarship<br>
@date: December 2020

'''
Trains classifiers to predict whether an article is about a given perspective in org. science. To train the classifiers, uses preliminary labeled articles, broken down as follows: 
Cultural: 105 yes, 209 no
Relational: 92 yes, 230 no
Demographic: 77 yes, 249 no
Compares f1_weighted scores of four model structures using 10-Fold Cross Validation: Logistic regression, SVM, Naive Bayes, and Decision Tree. Oversamples training data to .7 (7:10 minority:majority class).
'''

## Initialize

In [1]:
######################################################
# Import libraries
######################################################

import pandas as pd
import numpy as np
import re
from collections import Counter
from datetime import date
import os

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')

stemmer = WordNetLemmatizer()

from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt

import joblib
import csv

from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split, KFold

# !pip install imblearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

import sys; sys.path.insert(0, "../preprocess/") # For loading functions from files in other directory
from quickpickle import quickpickle_dump, quickpickle_load # custom scripts for quick saving & loading to pickle format

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
######################################################
# Define filepaths
######################################################

thisday = date.today().strftime("%m%d%y")

cwd = os.getcwd()
root = str.replace(cwd, 'classification/modeling', '')

# Directory for prepared data and trained models: save files here
data_fp = root + 'classification/data/'
model_fp = root + 'classification/models/'

# Current article lists
article_list_fp = data_fp + 'filtered_length_index.csv' # Filtered index of research articles
article_paths_fp = data_fp + 'filtered_length_article_paths.csv' # List of article file paths

# Preprocessed training data
cult_labeled_fp = data_fp + 'training_cultural_preprocessed_121120.pkl'
relt_labeled_fp = data_fp + 'training_relational_preprocessed_121120.pkl'
demog_labeled_fp = data_fp + 'training_demographic_preprocessed_121120.pkl'

# Model filepaths
cult_model_fp = model_fp + f'classifier_cult_{str(thisday)}.joblib'
relt_model_fp = model_fp + f'classifier_relt_{str(thisday)}.joblib'
demog_model_fp = model_fp + f'classifier_demog_{str(thisday)}.joblib'

# Vectorizers trained on hand-coded data (use to limit vocab of input texts)
cult_vec_fp = model_fp + 'vectorizer_cult_121120.joblib'
relt_vec_fp = model_fp + 'vectorizer_relt_121120.joblib'
demog_vec_fp = model_fp + 'vectorizer_demog_121120.joblib'

## Load & inspect data

In [3]:
cult_df = quickpickle_load(cult_labeled_fp)
relt_df = quickpickle_load(relt_labeled_fp)
demog_df = quickpickle_load(demog_labeled_fp)

cult_df.head(10)

,cultural_score,text
0,1.0,"[[journal, of, managerial, issues, vol], [xxii..."
1,1.0,"[[organization, ht, icna, vol], [no], [may, ju..."
2,1.0,"[[from, fiefs, to, clans, and, network, capita..."
3,1.0,"[[the, collective, strategy, framework, an, ap..."
4,1.0,"[[manag, int, rev, doi, sl, research, article,..."
5,1.0,"[[int], [studies, ofmgt], [amp, org, vol], [no..."
6,1.0,"[[linking, organizational, values, to, relatio..."
7,1.0,"[[journal, of, organizational, behavior, organ..."
8,1.0,"[[®, academy, oí, management, learning, amp, e..."
9,1.0,"[[strategie, management, journal, strat], [mgm..."


In [4]:
# Check score distribution across classes
print(cult_df.groupby('cultural_score').size())
print()
print(relt_df.groupby('relational_score').size())
print()
print(demog_df.groupby('demographic_score').size())

cultural_score
0.0    209
0.5     12
1.0    105
dtype: int64

relational_score
0.0    229
0.5      7
1.0     92
dtype: int64

demographic_score
0.0    248
0.5      5
1.0     77
dtype: int64


In [5]:
# Drop unsure cases: where X_score = 0.5
drop_unsure = True

if drop_unsure:
    cult_df_yes = cult_df[cult_df['cultural_score'] == 1.0]
    cult_df_no = cult_df[cult_df['cultural_score'] == 0.0]
    cult_df = pd.concat([cult_df_yes, cult_df_no])
    
    relt_df_yes = relt_df[relt_df['relational_score'] == 1.0]
    relt_df_no = relt_df[relt_df['relational_score'] == 0.0]
    relt_df = pd.concat([relt_df_yes, relt_df_no])
    
    demog_df_yes = demog_df[demog_df['demographic_score'] == 1.0]
    demog_df_no = demog_df[demog_df['demographic_score'] == 0.0]
    demog_df = pd.concat([demog_df_yes, demog_df_no])

### Check vocab size and frequent words

In [6]:
def collect_article_tokens(article, return_string=False):
    '''
    Collects words from already-tokenized sentences representing each article.
    
    Args:
        article: list of lists of words (each list is a sentence)
        return_string: whether to return single, long string representing article
    Returns:
        tokens: string if return_string, else list of tokens
    '''
    
    tokens = [] # initialize
    
    if return_string:
        for sent in article:
            sent = ' '.join(sent) # make sentence into a string
            tokens.append(sent) # add sentence to list of sentences
        tokens = ' '.join(tokens) # join sentences into string
        return tokens # return string
    
    else:
        for sent in article:
            tokens += [word for word in sent] # add each word to list of tokens
        return tokens # return list of tokens

# For capturing word frequencies, add all words from each article to single, shared list (can't use this to create models)
cult_tokens = []; cult_df['text'].apply(lambda article: cult_tokens.extend([word for word in collect_article_tokens(article)]))
relt_tokens = []; relt_df['text'].apply(lambda article: relt_tokens.extend([word for word in collect_article_tokens(article)]))
demog_tokens = []; demog_df['text'].apply(lambda article: demog_tokens.extend([word for word in collect_article_tokens(article)]))
print()

In [7]:
# Look at size of vocabulary and most frequent words
tokens = (cult_tokens + relt_tokens) + demog_tokens
print('Vocab size:', len(set(tokens)))
print()

# Check out most frequent words in labeled texts
freq = Counter(tokens)
print('20 most frequent words in labeled articles:')
freq.most_common(20)

Vocab size: 75614

20 most frequent words in labeled articles:


[('the', 541746),
 ('of', 388052),
 ('and', 300309),
 ('in', 210253),
 ('to', 197892),
 ('that', 106610),
 ('is', 95025),
 ('for', 85691),
 ('as', 69812),
 ('on', 57671),
 ('are', 57133),
 ('with', 53278),
 ('by', 51497),
 ('this', 48992),
 ('be', 42995),
 ('oasis', 40297),
 ('or', 38481),
 ('it', 36482),
 ('from', 36262),
 ('an', 35447)]

### Check frequent sentences (to improve cleaning)

In [8]:
# Add sentences from each article to empty list:
cult_sents = []; cult_df['text'].apply(lambda article: cult_sents.extend([' '.join([word for word in sent]) for sent in article]))
relt_sents = []; relt_df['text'].apply(lambda article: relt_sents.extend([' '.join([word for word in sent]) for sent in article]))
demog_sents = []; demog_df['text'].apply(lambda article: demog_sents.extend([' '.join([word for word in sent]) for sent in article]))

sents = (cult_sents + relt_sents) + demog_sents
print('Number of sentences:', len(sents))
print()

# Check out most frequent sentences in labeled texts
freq = Counter(sents)
print('20 most frequent sentences in labeled articles:')
freq.most_common(20)

Number of sentences: 472263

20 most frequent sentences in labeled articles:


[('valign bottom oasis entry oasis entry colname colsep rowsep align char char',
  3751),
 ('pp', 1425),
 ('oasis entry oasis entry colname colsep rowsep align char char', 1271),
 ('no', 1100),
 ('oasis entry colname colsep rowsep align char char', 1077),
 ('american sociological review', 1041),
 ('american journal of sociology', 1000),
 ('administrative science quarterly', 941),
 ('valign bottom oasis entry colname colsep rowsep align char char', 938),
 ('sci', 491),
 ('academy of management journal', 444),
 ('chicago university of chicago press', 443),
 ('organ', 365),
 ('colsep rowsep oasis entry align char char', 360),
 ('academy of management review', 359),
 ('ed', 322),
 ('new york oxford university press', 317),
 ('new york free press', 307),
 ('oasis entry oasis entry colsep rowsep align char char', 296),
 ('cambridge ma harvard university press', 291)]

### Load and apply text vectorizers

In [9]:
# Collect article tokens into a list of strings, each string an article
cult_tokens = []; cult_df['text'].apply(lambda article: cult_tokens.append(collect_article_tokens(article, return_string = True)))
relt_tokens = []; relt_df['text'].apply(lambda article: relt_tokens.append(collect_article_tokens(article, return_string = True)))
demog_tokens = []; demog_df['text'].apply(lambda article: demog_tokens.append(collect_article_tokens(article, return_string = True)))
print()

In [13]:
# Define stopwords used by JSTOR
jstor_stopwords = set(["a", "an", "and", "are", "as", "at", "be", "but", "by", "for", "if", "in", "into", "is", "it", "no", "not", "of", "on", "or", "such", "that", "the", "their", "then", "there", "these", "they", "this", "to", "was", "will", "with"])

# Uses TFIDF weighted DTM because results in better classifier accuracy than unweighted
cult_vectorizer = joblib.load(cult_vec_fp, "r+")
X_cult = cult_vectorizer.fit_transform(cult_tokens)
print('Number of features in cultural vectorizer:', len(cult_vectorizer.get_feature_names()))
print(cult_vectorizer.get_feature_names()[::1000]) # get every 1000th word
print()

relt_vectorizer = joblib.load(relt_vec_fp, "r+")
X_relt = relt_vectorizer.fit_transform(relt_tokens)
print('Number of features in relational vectorizer:', len(relt_vectorizer.get_feature_names()))
print(relt_vectorizer.get_feature_names()[::1000]) # get every 1000th word
print()

demog_vectorizer = joblib.load(demog_vec_fp, "r+")
X_demog = demog_vectorizer.fit_transform(demog_tokens)
print('Number of features in demographic vectorizer:', len(demog_vectorizer.get_feature_names()))
print(demog_vectorizer.get_feature_names()[::1000]) # get every 1000th word

Number of features in cultural vectorizer: 62115
['aa', 'afixed', 'andp', 'artes', 'backtocauses', 'beroepsarbeid', 'bottles', 'byzantine', 'cer', 'clemen', 'comports', 'conyon', 'culinary', 'deflation', 'dibia', 'divoky', 'eca', 'endangers', 'ethnonationalism', 'falbe', 'flavors', 'fuelwood', 'giraud', 'guigni', 'hennis', 'hsd', 'impinging', 'inome', 'ipb', 'journalof', 'kohl', 'leaking', 'loans', 'makrellag', 'mcluhan', 'ministrative', 'muisic', 'nerve', 'notable', 'ool', 'owning', 'percap', 'plummer', 'premised', 'ps', 'rationalities', 'reincarnated', 'revamping', 'ruef', 'schumpeterian', 'shaper', 'sleek', 'spouses', 'studentized', 'symbol', 'testimony', 'tongkonan', 'turation', 'unify', 'vaughan', 'wasp', 'workweeks', 'íí']

Number of features in relational vectorizer: 62933
['aa', 'afred', 'andre', 'artifacts', 'bacteriophage', 'berscheid', 'bottom', 'búngalo', 'certifying', 'clientage', 'compromise', 'copenhagen', 'current', 'delegitimated', 'difficultto', 'dodüü', 'eder', 'enit

## Setup for modeling

In [14]:
######################################################
# Balance x_train, y_train
######################################################

def resample_data(X_train, Y_train, undersample = False, sampling_ratio = 1.0):
    """
    Args:
        X_train: X training data
        Y_train: Y training data
        undersample: boolean for over or undersampling
        sampling_ratio: ratio of minority to majority class
        
        archived/not used:
        sampling_strategy: strategy for resampled distribution
            if oversample: 'majority' makes minority = to majority
            if undersample: 'minority' makes majority = to minority
            
    Returns:
        X_balanced: predictors at balanced ratio
        Y_balanced: outcomes at balanced ratio
    """
    
    if undersample == True:
        undersample = RandomUnderSampler(sampling_strategy=sampling_ratio)
        X_balanced, Y_balanced = undersample.fit_resample(X_train, Y_train)
    else:
        oversample = RandomOverSampler(sampling_strategy=sampling_ratio)
        X_balanced, Y_balanced = oversample.fit_resample(X_train, Y_train)
    
    print(f'Y_train: {Counter(Y_train)}\nY_resample: {Counter(Y_balanced)}')
    
    return X_balanced, Y_balanced

In [15]:
######################################################
# k-fold cross validation for model evaluation
######################################################

# Define test options for k-fold CV
num_folds = 10 
seed = 3
scoring='f1_weighted' # set scoring metric (not used here)

def show_kfold_output(models, 
                      X, 
                      Y, 
                      num_folds = num_folds, 
                      random_state = seed, 
                      shuffle = True):
    '''
    Estimates the accuracy of different model algorithms, adds results to a results array and returns.
    Prints the accuracy results: averages and std.
    Uses cross_val_predict, which unlike cross_val_score cannot define scoring option/evaluation metric.
    
    Args:
        models: list of (name, model) tuples
        X: predictors
        Y: outcomes
        num_folds: Split data randomly into num_folds parts: (num_folds-1) for training, 1 for scoring
        random_state: seed
        shuffle: 
    
    Returns:
        results: list of model results
        names: list of model names (matches results)
        
    Source: 
        https://stackoverflow.com/questions/40057049/using-confusion-matrix-as-scoring-metric-in-cross-validation-in-scikit-learn
    '''
    
    results = []
    names = []
    
    for name, model in models:
        # Setup model options
        kfold = KFold(
            n_splits=num_folds, 
            random_state=seed, 
            shuffle=True)
        
        # Get kfold results
        cv_results = cross_val_predict(
            model, 
            X, 
            Y, 
            cv=kfold, 
            #scoring=scoring, 
            n_jobs=-1) # use all cores = faster
        
        # Add results and name of each algorithm to the model array
        results.append(cv_results)
        names.append(name)
        
        # Print results
        print(f'{name}:')
        print()
        print(f'Mean (std):\t {round(cv_results.mean(),4)} ({round(cv_results.std(),4)})')
        print(f'Accuracy:\t', {round(accuracy_score(Y_balanced, cv_results)), 4})
        print()
        print('Confusion matrix:\n', confusion_matrix(Y_balanced, cv_results))
        print()
        print('Report:\n', classification_report(Y_balanced, cv_results))
        print()
        
    # Return arrays
    return results, names

## Evaluate algorithms: Cultural perspective

In [16]:
######################################################
# Prepare training and validation data
######################################################

# Separate training and final validation data set. First remove class
# label from data (X). Setup target class (Y)
# Then make the validation set 10% of the entire
# set of labeled data (X_validate, Y_validate)

cult_df = cult_df[['text', 'cultural_score']]
print("Number of cases:", str(X_cult.shape[0]))

valueArray = cult_df.values
Y = valueArray[:,1]
Y = Y.astype('float')
print("Number of codes (should match):", str(len(Y)))

test_size = 0.2
seed = 3
X_train, X_validate, Y_train, Y_validate = train_test_split(
    X_cult, 
    Y, 
    test_size=test_size, 
    random_state=seed)

print(f'Y_train Distribution: {Counter(Y_train).most_common()}')

Number of cases: 314
Number of codes (should match): 314
Y_train Distribution: [(0.0, 170), (1.0, 81)]


In [17]:
######################################################
# Oversample to desirable ratio
######################################################

# Use these settings here and below
sampling_ratio = 1.0 # ratio of minority to majority cases
undersample = False # whether to undersample or oversample

X_balanced, Y_balanced = resample_data(
    X_train, 
    Y_train, 
    undersample=undersample, 
    sampling_ratio=sampling_ratio)

Y_train: Counter({0.0: 170, 1.0: 81})
Y_resample: Counter({0.0: 170, 1.0: 170})


In [18]:
######################################################
# Use different algorithms to build models
######################################################

models = []
models.append(('K-Nearest Neighbors (KNN)', KNeighborsClassifier()))
models.append(('Random Forest (RF)', RandomForestClassifier(random_state=seed)))
models.append(('Decision Tree (DT)', DecisionTreeClassifier(random_state=seed)))
models.append(('Multinomial Naive Bayes (MNB)', MultinomialNB()))
models.append(('Logistic Regression (LR)', LogisticRegression(random_state=seed)))
models.append(('Support Vector Machine (SVM)', SVC(gamma='auto')))

# Evaluate algorithms using 10-fold cross validation
results, names = show_kfold_output(models=models, 
                                   X=X_balanced, 
                                   Y=Y_balanced)

K-Nearest Neighbors (KNN):

Mean (std):	 0.6294 (0.483)
Accuracy:	 {1.0, 4}

Confusion matrix:
 [[117  53]
 [  9 161]]

Report:
               precision    recall  f1-score   support

         0.0       0.93      0.69      0.79       170
         1.0       0.75      0.95      0.84       170

    accuracy                           0.82       340
   macro avg       0.84      0.82      0.81       340
weighted avg       0.84      0.82      0.81       340


Random Forest (RF):

Mean (std):	 0.4882 (0.4999)
Accuracy:	 {1.0, 4}

Confusion matrix:
 [[163   7]
 [ 11 159]]

Report:
               precision    recall  f1-score   support

         0.0       0.94      0.96      0.95       170
         1.0       0.96      0.94      0.95       170

    accuracy                           0.95       340
   macro avg       0.95      0.95      0.95       340
weighted avg       0.95      0.95      0.95       340


Decision Tree (DT):

Mean (std):	 0.5647 (0.4958)
Accuracy:	 {1.0, 4}

Confusion matrix:
 [[

In [27]:
######################################################
# Save best model
######################################################

rf_cult = RandomForestClassifier(random_state=seed).fit(X_balanced, Y_balanced)
joblib.dump(rf_cult, cult_model_fp)

['/home/jovyan/work/classification/models/classifier_cult_121120.joblib']

## Evaluate algorithms: Relational perspective

In [28]:
######################################################
# Prepare training and validation data
######################################################

# Separate training and final validation data set. First remove class
# label from data (X). Setup target class (Y)
# Then make the validation set 10% of the entire
# set of labeled data (X_validate, Y_validate)

relt_df = relt_df[['text', 'relational_score']]
print("Number of cases:", str(X_relt.shape[0]))

valueArray = relt_df.values
Y = valueArray[:,1]
Y = Y.astype('float')
print("Number of codes (should match):", str(len(Y)))

test_size = 0.2
seed = 3
X_train, X_validate, Y_train, Y_validate = train_test_split(X_relt, Y, test_size=test_size, random_state=seed)

print(f'Y_train Distribution: {Counter(Y_train).most_common()}')

Number of cases: 321
Number of codes (should match): 321
Y_train Distribution: [(0.0, 185), (1.0, 71)]


In [29]:
######################################################
# Oversample to desirable ratio
######################################################

X_balanced, Y_balanced = resample_data(
    X_train, Y_train, 
    undersample=undersample, 
    sampling_ratio=sampling_ratio)

Y_train: Counter({0.0: 185, 1.0: 71})
Y_resample: Counter({0.0: 185, 1.0: 185})


In [30]:
######################################################
# Use different algorithms to build models
######################################################

models = []
models.append(('K-Nearest Neighbors (KNN)', KNeighborsClassifier()))
models.append(('Random Forest (RF)', RandomForestClassifier(random_state=seed)))
models.append(('Decision Tree (DT)', DecisionTreeClassifier(random_state=seed)))
models.append(('Multinomial Naive Bayes (MNB)', MultinomialNB()))
models.append(('Logistic Regression (LR)', LogisticRegression(random_state=seed)))
models.append(('Support Vector Machine (SVM)', SVC(gamma='auto')))

# Evaluate algorithms using 10-fold cross validation
results, names = show_kfold_output(models=models, 
                                   X=X_balanced, 
                                   Y=Y_balanced)

K-Nearest Neighbors (KNN):

Mean (std):	 0.5973 (0.4904)
Accuracy:	 {1.0, 4}

Confusion matrix:
 [[145  40]
 [  4 181]]

Report:
               precision    recall  f1-score   support

         0.0       0.97      0.78      0.87       185
         1.0       0.82      0.98      0.89       185

    accuracy                           0.88       370
   macro avg       0.90      0.88      0.88       370
weighted avg       0.90      0.88      0.88       370


Random Forest (RF):

Mean (std):	 0.4757 (0.4994)
Accuracy:	 {1.0, 4}

Confusion matrix:
 [[184   1]
 [ 10 175]]

Report:
               precision    recall  f1-score   support

         0.0       0.95      0.99      0.97       185
         1.0       0.99      0.95      0.97       185

    accuracy                           0.97       370
   macro avg       0.97      0.97      0.97       370
weighted avg       0.97      0.97      0.97       370


Decision Tree (DT):

Mean (std):	 0.5297 (0.4991)
Accuracy:	 {1.0, 4}

Confusion matrix:
 [

In [31]:
######################################################
# Save best model
######################################################

rf_relt = RandomForestClassifier(random_state=seed).fit(X_balanced, Y_balanced)
joblib.dump(rf_relt, relt_model_fp)

['/home/jovyan/work/classification/models/classifier_relt_121120.joblib']

## Evaluate algorithms: Demographic perspective

In [32]:
######################################################
# Prepare training and validation data
######################################################

# Separate training and final validation data set. First reltove class
# label from data (X). Setup target class (Y)
# Then make the validation set 10% of the entire
# set of labeled data (X_validate, Y_validate)

demog_df = demog_df[['text', 'demographic_score']]
print("Number of cases:", str(X_demog.shape[0]))

valueArray = demog_df.values
Y = valueArray[:,1]
Y = Y.astype('float')
print("Number of codes (should match):", str(len(Y)))

test_size = 0.2
seed = 3
X_train, X_validate, Y_train, Y_validate = train_test_split(X_demog, Y, test_size=test_size, random_state=seed)

print(f'Y_train Distribution: {Counter(Y_train).most_common()}')

Number of cases: 325
Number of codes (should match): 325
Y_train Distribution: [(0.0, 200), (1.0, 60)]


In [33]:
######################################################
# Oversample to desirable ratio
######################################################

X_balanced, Y_balanced = resample_data(
    X_train, Y_train, 
    undersample=undersample, 
    sampling_ratio=sampling_ratio)

Y_train: Counter({0.0: 200, 1.0: 60})
Y_resample: Counter({1.0: 200, 0.0: 200})


In [34]:
######################################################
# Use different algorithms to build models
######################################################

models = []
models.append(('K-Nearest Neighbors (KNN)', KNeighborsClassifier()))
models.append(('Random Forest (RF)', RandomForestClassifier(random_state=seed)))
models.append(('Decision Tree (DT)', DecisionTreeClassifier(random_state=seed)))
models.append(('Multinomial Naive Bayes (MNB)', MultinomialNB()))
models.append(('Logistic Regression (LR)', LogisticRegression(random_state=seed)))
models.append(('Support Vector Machine (SVM)', SVC(gamma='auto')))

# Evaluate algorithms using 10-fold cross validation
results, names = show_kfold_output(models=models, 
                                   X=X_balanced, 
                                   Y=Y_balanced)

K-Nearest Neighbors (KNN):

Mean (std):	 0.665 (0.472)
Accuracy:	 {1.0, 4}

Confusion matrix:
 [[134  66]
 [  0 200]]

Report:
               precision    recall  f1-score   support

         0.0       1.00      0.67      0.80       200
         1.0       0.75      1.00      0.86       200

    accuracy                           0.83       400
   macro avg       0.88      0.83      0.83       400
weighted avg       0.88      0.83      0.83       400


Random Forest (RF):

Mean (std):	 0.5025 (0.5)
Accuracy:	 {1.0, 4}

Confusion matrix:
 [[199   1]
 [  0 200]]

Report:
               precision    recall  f1-score   support

         0.0       1.00      0.99      1.00       200
         1.0       1.00      1.00      1.00       200

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00      1.00      1.00       400


Decision Tree (DT):

Mean (std):	 0.5375 (0.4986)
Accuracy:	 {1.0, 4}

Confusion matrix:
 [[179 

In [35]:
######################################################
# Save best model
######################################################

rf_demog = RandomForestClassifier(random_state=seed).fit(X_balanced, Y_balanced)
joblib.dump(rf_demog, demog_model_fp)

['/home/jovyan/work/classification/models/classifier_demog_121120.joblib']